In [2]:
import torch
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print(x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


In [31]:
from huggingface_hub import notebook_login

notebook_login()



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Login successful
Your token has been saved to /Users/nateweinstein/.huggingface/token
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the def

In [9]:
# https://huggingface.co/docs/transformers/tasks/summarization
# import transformers
from datasets import load_dataset
billsum = load_dataset("billsum", split="ca_test")

Found cached dataset billsum (/Users/nateweinstein/.cache/huggingface/datasets/billsum/default/3.0.0/75cf1719d38d6553aa0e0714c393c74579b083ae6e164b2543684e3e92e0c4cc)


AttributeError: 'Dataset' object has no attribute 'truncate'

In [11]:
billsum = billsum.train_test_split(test_size=0.2)

In [ ]:
# print(billsum['train'][2])
# import pandas as pd
# data = pd.DataFrame(data=billsum['train'])
# data.describe()

In [12]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-small")


/Users/nateweinstein/opt/miniconda3/envs/summarization_poc/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5_fast.py:156: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [38]:
prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=850, truncation=True)
    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_billsum = billsum.map(preprocess_function, batched=True)

from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer)


In [39]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer)


import evaluate
rouge = evaluate.load("rouge")


In [40]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}
    

In [41]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

loading configuration file config.json from cache at /Users/nateweinstein/.cache/huggingface/hub/models--t5-small/snapshots/3479082dc36f8a4730936ef1c9b88cd8b0835c53/config.json
Model config T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngra

In [42]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_awesome_billsum_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
#     fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/Users/nateweinstein/Projects/hugging_face/intro/my_awesome_billsum_model is already a clone of https://huggingface.co/natedog/my_awesome_billsum_model. Make sure you pull the latest changes with `repo.git_pull()`.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: title, summary, text. If title, summary, text are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/Users/nateweinstein/opt/miniconda3/envs/summarization_poc/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 989
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 248
  Number of trainable parameters = 60506624


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,2.811071,0.128400,0.037200,0.107300,0.107000,19.000000
2,No log,2.598755,0.135800,0.046800,0.113100,0.113100,19.000000
3,No log,2.534993,0.136900,0.046600,0.111800,0.111900,19.000000
4,No log,2.517903,0.137500,0.046800,0.112900,0.112900,19.000000


The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: title, summary, text. If title, summary, text are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 248
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: title, summary, text. If title, summary, text are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 248
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: title, summary, text. If title, summary, text are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Ru

TrainOutput(global_step=248, training_loss=3.0268109229303177, metrics={'train_runtime': 11488.9092, 'train_samples_per_second': 0.344, 'train_steps_per_second': 0.022, 'total_flos': 1070824333246464.0, 'train_loss': 3.0268109229303177, 'epoch': 4.0})

In [43]:
trainer.push_to_hub()

Saving model checkpoint to my_awesome_billsum_model
Configuration saved in my_awesome_billsum_model/config.json
Model weights saved in my_awesome_billsum_model/pytorch_model.bin
tokenizer config file saved in my_awesome_billsum_model/tokenizer_config.json
Special tokens file saved in my_awesome_billsum_model/special_tokens_map.json


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Several commits (3) will be pushed upstream.
The progress bars may be unreliable.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

batch response: Authorization error.
error: failed to push some refs to 'https://huggingface.co/natedog/my_awesome_billsum_model'

error: failed to push some refs to 'https://huggingface.co/natedog/my_awesome_billsum_model'



OSError: batch response: Authorization error.
error: failed to push some refs to 'https://huggingface.co/natedog/my_awesome_billsum_model'


In [1]:
from transformers import pipeline

summarizer = pipeline("summarization", model="/Users/nateweinstein/Projects/hugging_face/intro/my_awesome_billsum_model")

In [48]:
# text='There’s a growing appetite for alternative search engines. At least, that’s the crux of the argument Richard Socher, the former chief scientist at Salesforce, likes to make. In 2020, Socher co-founded You, a search engine that uses AI to understand search queries, rank the results and parse the queries into different languages (including programming languages). You summarizes information from across the web and offers built-in apps, like search tools for Twitter, that allow users to complete tasks without having to leave the results page. It seems there’s some truth to his words. Socher claims that You has hundreds of thousands of users, with 70% growth in sign-ups last month and 30% growth in unique searches month over month. While that pales in comparison to the world’s most popular search engines (i.e., Google, Bing), which have hundreds of millions of users, Socher draws attention to You’s retention rate. Fifty percent of people who set You as their default search engine continue to use it after the fact. The numbers are to investors’ liking. Today, You closed a $25 million funding round led by Radical Ventures with participation from Salesforce CEO Marc Benioff’s Time Ventures, Breyer Capital, Norwest Venture Partners and Day One Ventures. It brings the startup’s total raised to $45 million, which Socher said will be put toward developing “premium features” and collaborating with outside developers to “show more useful actionable apps” in You’s search results page. “We envision You.com becoming a search platform that is open and allows others to build on top of all of the search technology that we’ve created. Data transparency, user customization, summarization, privacy and state of the art search are the foundation of our platform, and we beat Google in the long run by empowering the world to build the next search experience together,” Socher told TechCrunch in an email interview. “Google is a monolithic, monopolistic search engine that is closed and has ultimately weaponized AI against users for the sake of serving its true purpose: advertising. We are building You.com as a search platform that is open and emphasizes directly serving user needs with You.com apps instead of bombarding people with ads.” Socher previously founded MetaMind, an AI startup that was acquired by Salesforce in 2016. While at Salesforce, Socher helped to build the company’s sprawling Einstein AI platform and worked with Bryan McCann, You’s second co-founder, on natural language processing research.   Unsurprisingly given Socher’s data science background, You sprinkles AI-enabled features liberally throughout the search experience. A capability newly launched today, YouCode can generate code along the lines of GitHub’s Copilot based on a search query. And the recently debuted YouWrite, powered by OpenxAI’s GPT-3, can be prompted to write essays, blog posts and even boilerplate letters.'

In [56]:
!pip install nltk
import nltk
nltk.download('punkt')



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nateweinstein/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [57]:
# nltk_tokens = nltk.word_tokenize(text)
# print(nltk_tokens)
# print("Number of Words: " , len(nltk_tokens))

['There', '’', 's', 'a', 'growing', 'appetite', 'for', 'alternative', 'search', 'engines', '.', 'At', 'least', ',', 'that', '’', 's', 'the', 'crux', 'of', 'the', 'argument', 'Richard', 'Socher', ',', 'the', 'former', 'chief', 'scientist', 'at', 'Salesforce', ',', 'likes', 'to', 'make', '.', 'In', '2020', ',', 'Socher', 'co-founded', 'You', ',', 'a', 'search', 'engine', 'that', 'uses', 'AI', 'to', 'understand', 'search', 'queries', ',', 'rank', 'the', 'results', 'and', 'parse', 'the', 'queries', 'into', 'different', 'languages', '(', 'including', 'programming', 'languages', ')', '.', 'You', 'summarizes', 'information', 'from', 'across', 'the', 'web', 'and', 'offers', 'built-in', 'apps', ',', 'like', 'search', 'tools', 'for', 'Twitter', ',', 'that', 'allow', 'users', 'to', 'complete', 'tasks', 'without', 'having', 'to', 'leave', 'the', 'results', 'page', '.', 'It', 'seems', 'there', '’', 's', 'some', 'truth', 'to', 'his', 'words', '.', 'Socher', 'claims', 'that', 'You', 'has', 'hundreds'

In [43]:
from tinydb import TinyDB, Query
from lib.get_web import strip_tags

# Set up a temp DB for easier writing
db = TinyDB('db/database.json')

def search_word_in_string(word, string, size):
    index = string.find(word)
    if index == -1:
        return "Word not found in the string."
    start = max(0, index - size // 2)
    end = min(len(string), start + size)
    return string[start:end]


In [62]:
author = 'Chris Dixon'
term = 'climate'

entry = Query()
authorArticles = db.search(entry.author.matches('.*'+author+'.*') & entry.body.search('.*'+term+'.*'))
relevant = ''

for art in authorArticles:
    cleaned = strip_tags(art['body'])
    relevant = relevant +' '+ search_word_in_string(term, cleaned, 7000)
print("Summarizing... Using "+str(len(authorArticles))+' articles')
summarizer(relevant)

Your max_length is set to 200, but you input_length is only 3. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)


Summarizing... Using 0 articles


[{'summary_text': 'a spokesman for the ssnns ffnn f fnr fps .'}]

In [49]:
summarizer(text)

[{'summary_text': 'In 2020, Socher co-founded You, a search engine that uses AI to understand search queries, rank the results and parse the queries into different languages (including programming languages) You has hundreds of thousands of users, with 70% growth in sign-ups last month and 30% growth in unique searches month over month. You closed a $25 million funding round with participation from Salesforce CEO Marc Benioff’s Time Ventures, Breyer Capital, Norwest Venture Partners and Day One Ventures.'}]